In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('./data/noise_removed.csv')

print(df.info())
print(df.describe())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1595 entries, 0 to 1594
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  1595 non-null   object 
 1   text                1595 non-null   object 
 2   target              1595 non-null   int64  
 3   special_char_count  1595 non-null   int64  
 4   special_char_ratio  1595 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 62.4+ KB
None
            target  special_char_count  special_char_ratio
count  1595.000000         1595.000000         1595.000000
mean      3.008777            6.310345            0.231351
std       2.003115            2.762150            0.091539
min       0.000000            1.000000            0.043478
25%       1.000000            4.000000            0.166667
50%       3.000000            6.000000            0.225806
75%       5.000000            8.000000            0.296296
max       6

In [ ]:
# # 정규식 패턴
# pattern = r"[^가-힣A-Z\u4E00-\u9FFF\s0-9\.%㎜㎡]"

# # 정규식을 사용해 text 열에서 노이즈 제거
# df['text'] = df['text'].apply(lambda x: re.sub(pattern, "", str(x)))

# print(df.head())

In [ ]:
def remove_eng_noise(text):
    # 대문자 두 글자 이상의 단어 또는 'vs'를 찾는 정규 표현식
    meaningful_pattern = r'[A-Z]{2,}|vs'
    
    # 결과를 담을 리스트
    cleaned_text = []

    current_word = ''
    for char in text:
        if re.match(r'[\uAC00-\uD7A3]', char):  # 한글인 경우
            if current_word:
                # 이전에 쌓인 단어가 있으면 의미 있는지 확인
                if re.match(meaningful_pattern, current_word):
                    cleaned_text.append(current_word)
                current_word = ''  # current_word 초기화
            cleaned_text.append(char)  # 한글 추가
        elif char.isalpha():  # 영어 알파벳인 경우
            current_word += char  # 영어 문자를 current_word에 추가
        else:  # 기타 문자는 그대로 유지
            if current_word:
                if re.match(meaningful_pattern, current_word):
                    cleaned_text.append(current_word)
                current_word = ''  # current_word 초기화
            cleaned_text.append(char)  # 기타 문자는 추가

    # 마지막으로 쌓인 current_word가 의미 있는지 확인
    if current_word and re.match(meaningful_pattern, current_word):
        cleaned_text.append(current_word)

    return ''.join(cleaned_text)


In [ ]:
df['text']= df['text'].apply(remove_eng_noise)

In [3]:
def clean_number(text):
    # 1. 숫자가 문자 사이에 있는 경우 숫자를 공백으로 대체
    text = re.sub(r'(?<=[a-zA-Z가-힣])\d+(?=[a-zA-Z가-힣])', ' ', text)

    # 2. 특수 기호가 '.'나 '…', '~'가 아닌 경우에 한해 숫자를 제거
    text = re.sub(r'(?<![.\u2026∼])([^\w\s.\u2026∼])\d+', r'\1', text)

    # 불필요한 공백 정리
    cleaned_text = re.sub(r'\s+', ' ', text).strip()
    
    return cleaned_text

In [4]:
df['text'] = df['text'].apply(clean_number)

In [5]:
display(df.head(30))

,ID,text,target,special_char_count,special_char_ratio
0,ynat-v1_train_00624,사건실를 라 극체O.월,0,14,0.560000
1,ynat-v1_train_02413,전W참 이 유객우려 . 개조,6,16,0.533333
2,ynat-v1_train_02275,더 대 김현권원대 L줄알았,2,15,0.500000
3,ynat-v1_train_02765,G 서울 곳 나객 적 귀거,3,15,0.500000
4,ynat-v1_train_01203,O통령후 싱턴으로 R與지부,2,15,0.500000
5,ynat-v1_train_02105,P홀 년H요서트Y 세NJ,0,16,0.500000
6,ynat-v1_train_01383,H마 대통령 후워싱턴무 개정,6,14,0.482759
7,ynat-v1_train_02013,에W성엘에스티에 규,5,11,0.478261
8,ynat-v1_train_01228,디딤. 대에도모신 O,5,9,0.473684
9,ynat-v1_train_00493,N민황4 두투이Z AM치 데뷔.,1,14,0.466667


In [7]:
# 결과를 새로운 CSV 파일로 저장
df.to_csv('./data/noise_removed_num_1600.csv', index=False)